This notebook walks through a self-contained implementation of
functorch, including support for both vjp and vmap combinators (using
PyTorch only to implement primitive tensor operations).  It follows
the tradition of
[Autodidax](https://jax.readthedocs.io/en/latest/autodidax.html) (a
pedagogical reimplementation of JAX, the library functorch is inspired
by) and [Simple
Autograd](https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC?usp=sharing)
(Zachary Devito's pedagogical reimplementation of autograd, which the
autograd system in this notebook is based off of.) You can [open this
file in
Colab](https://colab.research.google.com/github/albanD/subclass_zoo/blob/main/simple_functorch.ipynb)
and play around with the examples.

As a simplified implementation of functorch, this notebook also makes
it easier to investigate some more subtle aspects of how PyTorch's
native autograd system interacts with composable transforms.  In
particular, we will see that PyTorch's native implementation of double
backwards (which shares the same tape through multiple levels of
differentiation) differs from functorch's nested grad implementation
(which maintains a separate tape per level).

To get started, we replicate some of the data structures and helper functions
from Simple Autograd.

In [1]:
import torch
from torch import Tensor
from typing import List, NamedTuple, Callable, Dict, Optional


class TapeEntry(NamedTuple):
    # names of the inputs to the original computation
    inputs: List[str]
    # names of the outputs of the original computation
    outputs: List[str]
    # apply chain rule
    propagate: Callable[[List[Tensor]], List[Tensor]]


_name = 0


def fresh_name() -> str:
    """create a new unique name for a variable: v0, v1, v2"""
    global _name
    r = f"v{_name}"
    _name += 1
    return r

This is a little helper function for converting the dim argument in
sum into an explicit list of dimensions that will be reduced over.
It takes the dim of the tensor we are summing over and the dim
argument itself.

In [2]:
def sum_dims(*, input_dim, dim):
    if dim is None:
        return tuple(range(0, input_dim))
    elif isinstance(dim, int):
        return (dim,)
    else:
        return tuple(sorted(dim))

In Simple Autograd, we provided a Variable wrapper class which
provided a traditional Tensor style interface for our objects; in
functorch proper, objects are repeatedly wrapped in this way to
implement multipler layers of transformations.

In my opinion, this sort of wrapper makes it more difficult to
understand the flow of logic.  So in Simple Functorch, we take a
different approach: we won't make use of a wrapper class at all,
instead showing how to add it in the end as syntax sugar on top of our
system.

For debuggability purposes, however, it is nice to have a way to
identify variables by a human readable name.  We'll do this by setting
a t_name attribute on PyTorch tensors whenever we allocate a new
tensor.

In [3]:
def label(t: Tensor, name: str = None):
    if not hasattr(t, "t_name"):
        t.t_name = name or fresh_name()
    return t

So if we aren't going to have a wrapper around each tensor, how will
we actually implement our logic?  We will organize our various layers
of transformations as separate Dispatcher objects, which define
methods for performing operations on tensors, but are not Tensors
themselves.  For example, instead of defining Tensor.add(Tensor), we
will define Dispatcher.add(Tensor, Tensor).  If you are familiar with
historical ATen, in the original implementation of ATen, these
correspond to the CPUType/CUDAType/VariableType objects from that
implementation (this was replaced with the modern dispatcher as the
original vtable-based implementation did not support adding custom
operators.)

In [4]:
class Dispatcher:
    def mul(self, lhs, rhs):
        raise NotImplementedError

    def add(self, lhs, rhs):
        raise NotImplementedError

    # Sum has been generalized to take an optional dim argument, which we
    # will need for Batched tensors
    def sum(self, input, dim=None, name=None):
        raise NotImplementedError

    def expand(self, input, sizes):
        raise NotImplementedError

    # For closure under Batched tensors, we need these operations...
    def unsqueeze(self, input, dim):
        raise NotImplementedError

    def squeeze(self, input, dim):
        raise NotImplementedError

    # ...and we also need to overload the meaning of size/ones to
    # hide/reinsert batch dimensions
    def size(self, input):
        raise NotImplementedError

    def ones(self, size):
        raise NotImplementedError

    # For convenience, we provide dim, which just returns the length of
    # the sizes
    def dim(self, input):
        return len(self.size(input))

To start with, we can implement a backend dispatcher layer Torch,
which just forwards the operator calls to our underlying library PyTorch
(and ensures that all the allocated tensors are labeled with variable).  You could
also imagine replacing this with a Numpy backend or even a pure Python
variant (although this file is not currently setup to do so.)

In [ ]:
class Torch(Dispatcher):
    def mul(self, lhs, rhs):
        return label(torch.mul(lhs, rhs))

    def add(self, lhs, rhs):
        return label(torch.add(lhs, rhs))

    def sum(self, input, dim=None, name=None):
        if dim is None:
            return label(torch.sum(input), name)
        else:
            return label(torch.sum(input, dim), name)

    def expand(self, input, sizes):
        return label(input.expand(sizes))

    def unsqueeze(self, input, dim):
        return label(torch.unsqueeze(input, dim))

    def squeeze(self, input, dim):
        return label(torch.squeeze(input, dim))

    def size(self, input):
        # Return size a tuple for marginally more compact printing
        return tuple(input.size())

    def ones(self, size):
        return label(torch.ones(size))

Dispatcher layers are composable via object composition: we can
imagine a stack of dispatchers, each one calling into the next.
For example, the Logger dispatcher simply prints out what operation
was called on it, and then forwards on the operation to the inner
dispatcher.

In [5]:
class Logger(Dispatcher):
    def __init__(self, inner, *, name):
        self.inner = inner
        self.name = f"  {name}"

    def size(self, input):
        # don't log size calls
        return self.inner.size(input)

    def ones(self, size):
        r = self.inner.ones(size)
        print(f"{self.name} {r.t_name}: {self.size(r)} = ones({size})")
        return r

    def mul(self, lhs, rhs):
        r = self.inner.mul(lhs, rhs)
        if isinstance(rhs, float):
            print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs}")
        else:
            print(
                f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs.t_name}"
            )
        return r

    def add(self, lhs, rhs):
        r = self.inner.add(lhs, rhs)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} + {rhs.t_name}")
        return r

    def sum(self, input, dim=None, name=None):
        r = self.inner.sum(input, dim=dim, name=name)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.sum(dim={dim})")
        return r

    def unsqueeze(self, input, dim):
        r = self.inner.unsqueeze(input, dim)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.unsqueeze({dim})"
        )
        return r

    def squeeze(self, input, dim):
        r = self.inner.squeeze(input, dim)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.squeeze({dim})")
        return r

    def expand(self, input, sizes):
        r = self.inner.expand(input, sizes)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.expand({sizes})"
        )
        return r

Here is a simple example of using Logger and Torch together.  Whenever
we make calls to operations, we must do so via the Dispatcher object.
We will explicitly write out all of these calls before we add wrapper
class sugaring.

In [6]:
d = Logger(Torch(), name="Torch")
print(d.add(d.ones(2), d.ones(2)))

With the Dispatcher structure in hand, we are now in a good place to
port the autograd implementation from Simple Autograd into our new
framework.

In [ ]:
class Autograd(Dispatcher):
    # create_graph here corresponds to the create_graph kwarg in traditional
    # PyTorch, which controls whether or not the graph of the derivative
    # will be constructed, allowing computing higher order derivatives.
    # We will see that although create_graph=True allows Autograd to directly
    # support higher order derivatives, layering an Autograd to another
    # Autograd will also allow higher order derivatives.
    def __init__(self, inner, *, name="Autograd", create_graph: bool = False):
        self.inner = inner
        self.gradient_tape = []
        self.name = name
        self.create_graph = create_graph

    # create_graph controls where add/mul/etc calls from the backwards
    # propagators go: if you create_graph, they recursively call back to
    # the current Autograd dispatcher; otherwise they move on to the inner
    # layer.
    def backward_inner(self):
        if self.create_graph:
            return self
        else:
            return self.inner

    def mul(self, lhs, rhs):
        if isinstance(rhs, float) and rhs == 1.0:
            # peephole optimization
            return lhs

        # define forward
        # first, run the operation in the inner layer to get the initial
        # result
        r = self.inner.mul(lhs, rhs)
        # We directly implement printing here as it indicates whether or not
        # this operation was saved to the tape or not
        print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs.t_name}")

        # record what the inputs and outputs of the op were
        inputs = [lhs.t_name, rhs.t_name]
        outputs = [r.t_name]

        # define backprop
        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs

            dr_dlhs = rhs  # partial derivative of r = lhs*rhs
            dr_drhs = lhs  # partial derivative of r = lhs*rhs

            # chain rule propagation from outputs to inputs of multiply.
            # Notice that the propagation rule may itself call
            # other operations; depending on create_graph, they may
            # either go to self or self.inner; self.backward_inner()
            # controls which one we go to.
            dL_dlhs = self.backward_inner().mul(dL_dr, dr_dlhs)
            dL_drhs = self.backward_inner().mul(dL_dr, dr_drhs)
            dL_dinputs = [dL_dlhs, dL_drhs]
            return dL_dinputs

        # finally, we record the compute we did on the tape
        self.gradient_tape.append(
            TapeEntry(inputs=inputs, outputs=outputs, propagate=propagate)
        )
        return r

    # The rest of the implementations follow in the same way and can
    # be skipped

    def add(self, lhs, rhs):
        # Add follows a similar pattern to Mul, but it doesn't end up
        # capturing any variables.
        r = self.inner.add(lhs, rhs)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} + {rhs.t_name}")

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            dr_dlhs = 1.0
            dr_drhs = 1.0
            dL_dlhs = self.backward_inner().mul(dL_dr, dr_dlhs)
            dL_drhs = self.backward_inner().mul(dL_dr, dr_drhs)
            return [dL_dlhs, dL_drhs]

        self.gradient_tape.append(
            TapeEntry(
                inputs=[lhs.t_name, rhs.t_name], outputs=[r.t_name], propagate=propagate
            )
        )
        return r

    # Extended to handle dim argument for Batched (later)
    def sum(self, input: Tensor, dim=None, name: Optional[str] = None):
        r = self.inner.sum(input, dim=dim, name=name)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.sum(dim={dim})")

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            size = self.inner.size(input)
            res = dL_dr
            # Broadcast over all dimensions that were reduced over
            for i in sum_dims(input_dim=self.inner.dim(input), dim=dim):
                res = self.backward_inner().unsqueeze(res, i)
            return [self.backward_inner().expand(res, size)]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    # Unlike Simple Autograd, this expand requires the input to have
    # been unsqueezed before hand.  This lets us avoid having to do
    # at::sum_to for the nontrivial case (which is more complicated)
    def expand(self, input: Tensor, sizes: List[int]):
        assert self.inner.dim(input) == len(sizes)  # only works if dims match
        r = self.inner.expand(input, sizes)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.expand({sizes})"
        )

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            input_size = self.inner.size(input)
            dims = tuple(
                i for i in range(self.inner.dim(input)) if input_size[i] != sizes[i]
            )
            # We wanted a sum keepdim=True, but I didn't want to force
            # everyone to support it so manually unsqueeze
            res = self.backward_inner().sum(dL_dr, dims)
            for d in dims:
                res = self.backward_inner().unsqueeze(res, d)
            return [res]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    # Unsqueeze are required for sum backwards, and then squeeze is required
    # for closure.  Size needed for batched tensor to modify size.

    def size(self, input: Tensor):
        return self.inner.size(input)

    def squeeze(self, input: Tensor, dim):
        r = self.inner.squeeze(input, dim)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.squeeze(dim={dim})"
        )

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_outputs
            return [self.backward_inner().unsqueeze(dL_dr, dim)]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    def unsqueeze(self, input: Tensor, dim):
        r = self.inner.unsqueeze(input, dim)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.unsqueeze(dim={dim})"
        )

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            return [self.backward_inner().squeeze(dL_dr, dim)]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    def ones(self, size):
        return self.inner.ones(size)

    def grad(self, L, desired_results: List[Tensor]) -> List[Tensor]:
        # this map holds dL/dX for all values X
        dL_d: Dict[str, Tensor] = {}
        # It starts by initializing the 'seed' dL/dL, which is 1
        # TODO: indirect this via the backend
        dL_d[L.t_name] = self.inner.ones(())
        print(f"-- {self.name} d{L.t_name} -------")

        # look up dL_dentries. If a variable is never used to compute the loss,
        # we consider its gradient None, see the note below about zeros for more information.
        def gather_grad(entries: List[str]):
            return [dL_d[entry] if entry in dL_d else None for entry in entries]

        # propagate the gradient information backward
        for entry in reversed(self.gradient_tape):
            dL_doutputs = gather_grad(entry.outputs)
            if all(dL_doutput is None for dL_doutput in dL_doutputs):
                # optimize for the case where some gradient pathways are zero. See
                # The note below for more details.
                continue

            # perform chain rule propagation specific to each compute
            dL_dinputs = entry.propagate(dL_doutputs)

            # Accululate the gradient produced for each input.
            # Each use of a variable produces some gradient dL_dinput for that
            # use. The multivariate chain rule tells us it is safe to sum
            # all the contributions together.
            for input, dL_dinput in zip(entry.inputs, dL_dinputs):
                if input not in dL_d:
                    dL_d[input] = dL_dinput
                else:
                    dL_d[input] = self.backward_inner().add(dL_d[input], dL_dinput)

        # print some information to understand the values of each intermediate
        # for name, value in dL_d.items():
        #     print(f'{self.name} d{L.t_name}_d{name} = {value.t_name}')
        print(f"------------------------")

        return gather_grad(desired.t_name for desired in desired_results)

To calculate some simple gradients, we can compose Autograd with
Torch and get the result we expect.

In [7]:
torch.manual_seed(0)
a, b = label(torch.rand(4)), label(torch.rand(4))


def simple(d, a, b):
    t = d.add(a, b)
    return d.mul(t, b)


d = Autograd(Torch())

loss = simple(d, a, b)
da, db = d.grad(loss, [a, b])
print("da", da)
print("db", db)

Autograd v2: (4,) = v0 + v1
Autograd v3: (4,) = v2 * v1
-- Autograd dv3 -------
------------------------
da tensor([0.3074, 0.6341, 0.4901, 0.8964])
db tensor([1.1111, 2.0364, 1.0687, 1.9249])


To compute higher order gradients, we have two options.  First,
we can do traditional PyTorch style higher order differentiation
with `create_graph=True`, writing the backpropagation computations directly
into the tape so they can be further differentiated over.  This is also
what the original Simple Autograd implementation does.

In [8]:
d = Autograd(Torch(), create_graph=True)

# I slightly generalized this function so that it works for the next
# example; d2 is the dispatcher run on the first grad call, and d1 is
# for the second (we'll see why the numbers are inverted shortly).
def run_gradients(d2, d1):
    # our first loss
    L0 = d2.sum(simple(d2, a, b), name="L0")

    # compute derivatives of our inputs
    dL0_da, dL0_db = d2.grad(L0, [a, b])

    # In real code, how would we switch from executing from d2 to d1?
    # In functorch, the d2 dispatch calls would happen in the inside of
    # a higher-order grad() call; when we exit from this call, all
    # of the involved tensors are unwrapped.

    # now lets compute the L2 norm of our derivatives
    L1 = d1.sum(d1.add(d1.mul(dL0_da, dL0_da), d1.mul(dL0_db, dL0_db)), name="L1")

    # and take the gradient of that.
    # notice there are two losses involved1.
    dL1_da, dL1_db = d1.grad(L1, [a, b])
    return dL1_da, dL1_db


da, db = run_gradients(d, d)
print("da", da)
print("db", db)

Autograd v10: (4,) = v0 + v1
Autograd v11: (4,) = v10 * v1
Autograd L0: () = v11.sum(dim=None)
-- Autograd dL0 -------
Autograd v13: (1,) = v12.unsqueeze(dim=0)
Autograd v14: (4,) = v13.expand((4,))
Autograd v15: (4,) = v14 * v1
Autograd v16: (4,) = v14 * v10
Autograd v17: (4,) = v16 + v15
------------------------
Autograd v18: (4,) = v15 * v15
Autograd v19: (4,) = v17 * v17
Autograd v20: (4,) = v18 + v19
Autograd L1: () = v20.sum(dim=None)
-- Autograd dL1 -------
Autograd v22: (1,) = v21.unsqueeze(dim=0)
Autograd v23: (4,) = v22.expand((4,))
Autograd v24: (4,) = v23 * v17
Autograd v25: (4,) = v23 * v17
Autograd v26: (4,) = v24 + v25
Autograd v27: (4,) = v23 * v15
Autograd v28: (4,) = v23 * v15
Autograd v29: (4,) = v27 + v28
Autograd v30: (4,) = v29 + v26
Autograd v31: (4,) = v26 * v10
Autograd v32: (4,) = v26 * v14
Autograd v33: (4,) = v30 * v1
Autograd v34: (4,) = v30 * v14
Autograd v35: (4,) = v31 + v33
Autograd v36: () = v35.sum(dim=(0,))
Autograd v37: (1,) = v36.unsqueeze(dim=0)
A

Our second option is to follow functorch's implementation strategy, which
is to stack two Autograd dispatchers on top of each other.  Here, it is
not necessary to `create_graph=True`, because when the backpropagator forwards
to the inner dispatcher, it will record those operations on the tape too.
But if you look at the output, you will notice something very interesting:
the first portion of the tape is exactly replicated between Autograd1 and
Autograd2: we're duplicating the tape in this case!  So PyTorch's default
implementation of backwards is more efficient, because it avoids having to
record the tape twice (although this doesn't matter too much, because the
saved tensors themselves can be shared between the two tapes, so it is just
the operator graph that is duplicated.

This is our first example of using two dispatchers.  While we are
performing the inner grad, we perform our operations on the outer
dispatcher `d2`; after we are done with the inner grad we switch to
`d1`.  Intuitively, this corresponds from passing out of the inner
`grad` call to the outer `grad` call.

In [9]:
# turning off create_graph will impede us from seeing the logging lines for
# the second backwards, so we turn on logging for Torch to see them
d1 = Autograd(Logger(Torch(), name="Torch"), name="Autograd1", create_graph=False)
d2 = Autograd(d1, name="Autograd2", create_graph=False)

da, db = run_gradients(d2, d1)
print("da", da)
print("db", db)

  Torch v40: (4,) = v0 + v1
Autograd1 v40: (4,) = v0 + v1
Autograd2 v40: (4,) = v0 + v1
  Torch v41: (4,) = v40 * v1
Autograd1 v41: (4,) = v40 * v1
Autograd2 v41: (4,) = v40 * v1
  Torch L0: () = v41.sum(dim=None)
Autograd1 L0: () = v41.sum(dim=None)
Autograd2 L0: () = v41.sum(dim=None)
  Torch v42: () = ones(())
-- Autograd2 dL0 -------
  Torch v43: (1,) = v42.unsqueeze(0)
Autograd1 v43: (1,) = v42.unsqueeze(dim=0)
  Torch v44: (4,) = v43.expand((4,))
Autograd1 v44: (4,) = v43.expand((4,))
  Torch v45: (4,) = v44 * v1
Autograd1 v45: (4,) = v44 * v1
  Torch v46: (4,) = v44 * v40
Autograd1 v46: (4,) = v44 * v40
  Torch v47: (4,) = v46 + v45
Autograd1 v47: (4,) = v46 + v45
------------------------
  Torch v48: (4,) = v45 * v45
Autograd1 v48: (4,) = v45 * v45
  Torch v49: (4,) = v47 * v47
Autograd1 v49: (4,) = v47 * v47
  Torch v50: (4,) = v48 + v49
Autograd1 v50: (4,) = v48 + v49
  Torch L1: () = v50.sum(dim=None)
Autograd1 L1: () = v50.sum(dim=None)
  Torch v51: () = ones(())
-- Autogra

Under what situations might it be profitable to keep the two tapes separate?
One guess we might have is if there is another functional transformation
wedged between the two autograd transformations.  We would then expect the
backwards formula we save to be different between the two tapes.  To do this, I
first need to implement batched tensors.

One unusual thing about this implementation is that we do not need to wrap
tensors to change their sizes; instead, we just override the meaning of
size() on the dispatcher to hide batch dimensions.  One case we do not
exercise in this example is implicit broadcasting when you combine a tensor
that is not batched with a tensor that is batched: without wrappers, a user
must explicitly lift (e.g., unsqueeze and expand) tensors they wish to
replicate across the batch dimension.  The code below will blindly attempt to
reinterpret a tensor as a batched tensor, even when it may not make sense (if
there is a size mismatch, however, you will get an assert failure).  Similarly,
once you exit a vmap region, all previously vmap'ed tensors "magically" become
unbatched.  functorch did not pursue this implementation because at the time
Tensor.size() was not virtual and thus it was not possible to override (this
will be changing soon).

In [10]:
# This implementation of Batched only supports inserting a dimension
# at the very front
class Batched(Dispatcher):
    def __init__(self, inner, *, length, name="Batched"):
        self.inner = inner
        self.name = name
        self.length = length

    def size(self, input):
        sizes = self.inner.size(input)
        assert sizes[0] == self.length
        return sizes[1:]

    def ones(self, size):
        return self.inner.ones((self.length,) + size)

    def mul(self, lhs, rhs):
        assert self.inner.size(lhs)[0] == self.length
        if not isinstance(rhs, float):
            assert self.inner.size(rhs)[0] == self.length
        return self.inner.mul(lhs, rhs)

    def add(self, lhs, rhs):
        assert self.inner.size(lhs)[0] == self.length
        assert self.inner.size(rhs)[0] == self.length
        return self.inner.add(lhs, rhs)

    def sum(self, input, dim=None, name=None):
        # offset all the summed over dimensions by one
        assert self.inner.size(input)[0] == self.length
        dim = tuple(
            i + 1 for i in sum_dims(input_dim=self.inner.dim(input) - 1, dim=dim)
        )
        return self.inner.sum(input, dim, name=name)

    def expand(self, input, sizes):
        # offset sizes by one
        assert self.inner.size(input)[0] == self.length
        return self.inner.expand(input, (self.inner.size(input)[0],) + sizes)

    def squeeze(self, input, dim):
        # offset dim by one
        assert self.inner.size(input)[0] == self.length
        return self.inner.squeeze(input, dim + 1)

    def unsqueeze(self, input, dim):
        # offset dim by one
        assert self.inner.size(input)[0] == self.length
        return self.inner.unsqueeze(input, dim + 1)

In [11]:
# Our inputs are batched this time!
va, vb = label(torch.rand(2, 4)), label(torch.rand(2, 4))

d1 = Autograd(Torch(), name="Autograd1", create_graph=False)
d2 = Batched(d1, length=2, name="Batched2")
d3 = Autograd(d2, name="Autograd3", create_graph=False)


def run_batched_gradients(d3, d2, d1):
    # our first loss
    # we write the dimension we reduce on explicitly for clarity
    L0 = d3.sum(simple(d3, va, vb), dim=0, name="L0")

    # compute derivatives of our inputs
    dL0_da, dL0_db = d3.grad(L0, [va, vb])

    # now lets compute the L2 norm of our derivatives
    L1 = d1.sum(d1.add(d1.mul(dL0_da, dL0_da), d1.mul(dL0_db, dL0_db)), name="L1")

    # and take the gradient of that.
    # notice there are two losses involved1.
    dL1_da, dL1_db = d1.grad(L1, [va, vb])
    return dL1_da, dL1_db


dva, dvb = run_batched_gradients(d3, d2, d1)
print("va", va)
print("vb", vb)
print("dva", dva)
print("dvb", dvb)

Autograd1 v78: (2, 4) = v76 + v77
Autograd3 v78: (4,) = v76 + v77
Autograd1 v79: (2, 4) = v78 * v77
Autograd3 v79: (4,) = v78 * v77
Autograd1 L0: (2,) = v79.sum(dim=(1,))
Autograd3 L0: () = v79.sum(dim=0)
-- Autograd3 dL0 -------
Autograd1 v81: (2, 1) = v80.unsqueeze(dim=1)
Autograd1 v82: (2, 4) = v81.expand((2, 4))
Autograd1 v83: (2, 4) = v82 * v77
Autograd1 v84: (2, 4) = v82 * v78
Autograd1 v85: (2, 4) = v84 + v83
------------------------
Autograd1 v86: (2, 4) = v83 * v83
Autograd1 v87: (2, 4) = v85 * v85
Autograd1 v88: (2, 4) = v86 + v87
Autograd1 L1: () = v88.sum(dim=None)
-- Autograd1 dL1 -------
------------------------
va tensor([[0.4556, 0.6323, 0.3489, 0.4017],
        [0.0223, 0.1689, 0.2939, 0.5185]])
vb tensor([[0.6977, 0.8000, 0.1610, 0.2823],
        [0.6816, 0.9152, 0.3971, 0.8742]])
dva tensor([[3.7019, 4.4647, 1.3419, 1.9325],
        [2.7711, 3.9985, 2.1762, 4.5337]])
dvb tensor([[ 8.7992, 10.5293,  3.0059,  4.4296],
        [ 6.9054,  9.8274,  5.1466, 10.8156]])


To see that we have done this correctly, we could run the corresponding JAX:

```
from jax import grad, vmap
import jax.numpy as np

def simple(a, b):
  t = a + b
  return t * b

def L0(a, b):
  return np.sum(simple(a, b))

def L1(a, b):
  dL0_da, dL0_db = vmap(grad(L0, argnums=(0,1)), in_axes=0)(a, b)
  return (dL0_da * dL0_da + dL0_db * dL0_db).sum()

va = np.asarray([[0.4556, 0.6323, 0.3489, 0.4017],
        [0.0223, 0.1689, 0.2939, 0.5185]])
vb = np.asarray([[0.6977, 0.8000, 0.1610, 0.2823],
        [0.6816, 0.9152, 0.3971, 0.8742]])
dva, dvb = grad(L1, argnums=(0,1))(va, vb)
print("dva", dva)
print("dvb", dvb)
```

Looking over the output, the tapes look similar, but we can see that the sizes
and the arguments of the operations in question differ (after all, Autograd3 is
on the inside of the vmap, while Autograd1 is outside).  But it is still very
similar: we could imagine simply varying the dispatcher we use to process backwards
depending on when we are executing the tape.  In fact, this is exactly what an
initial, non-functorch implementation of PyTorch did to support per-sample
gradients.

Exercise: modify Autograd.grad to accept a dispatcher, and use that dispatcher
instead of self.backward_inner() when running propagator functions.  Then, rewrite
the above example so that it only has one level of Autograd:
Batched(Autograd(Torch(), create_graph=True)) and show you still get the same
result.

OK, so all of this dispatcher business is all nice and explicit, but
that's not what JAX/functorch's interface looks like.  How do we
bridge the gap?

TODO: write this

Epilogue: Why didn't I use inheritance?  In fact, in the first version
of this notebook, I did.  But self makes it easy to accidentally go
back to the "top" of the dispatch stack, which is typically not what
you want.

In [ ]:
print("All done")